# Preprocessing

In [1]:
import json
import csv
import os
import shutil
from tqdm import tqdm
import pandas as pd

In [5]:
data_root = "../materials"
working_root = "../working"
scratch_root = "../scratch"

os.makedirs(working_root, exist_ok=True)
os.makedirs(scratch_root, exist_ok=True)

**Unzip `x.listens.zst`**

In [ ]:
for i in range(12):
    src = os.path.join(data_root, f"{i + 1}.listens.zst")
    
    scratch_listen_1_compressed = shutil.copy2(src, scratch_root)
    !unzstd {scratch_listen_1_compressed}

## *User ID - MessyBrainz ID*

1. Collect `user_id` - `recording_msid` mappings from ListenBrainz dataset
2. Save to `userid-msid.csv`

**Write `userid-msid.csv`**

In [3]:
data = []
num_months = 3

for i in reversed(range(12)):
    with open(os.path.join(scratch_root, f"{i + 1}.listens")) as fp:
        for line in tqdm(fp):
            j = json.loads(line)
            user_id = j['user_id']
            recording_msid = j['recording_msid']
            data.append([user_id, recording_msid])

    if i <= 12 - num_months:
        break

with open(os.path.join(working_root, "userid-msid.csv"), "w", newline='', encoding='utf-8') as fp:
    w = csv.writer(fp)
    w.writerows(data)

7645939it [02:18, 55392.84it/s]
7342694it [02:12, 55231.99it/s]
7423517it [02:13, 55776.04it/s]


In [2]:
json = None

**Read `userid-msid.csv`**

In [4]:
data = pd.read_csv(os.path.join(working_root, "userid-msid.csv"), header=None)
data.columns = ["userid", "msid"]
data.head()

,userid,msid
0,21562,5371ebac-540e-4962-945f-cd583a81ae1f
1,30309,af3d7151-5bae-4c95-a398-e10bccc4220d
2,36460,0e97eba4-8776-4e1f-8573-025a82973d54
3,33714,16b99e42-c034-42ee-85c5-7fd0ba2b2c2d
4,19877,9bed3aaf-9fed-429e-8975-78b22603ad82


In [5]:
data.describe().loc['count']

userid    22412150.0
Name: count, dtype: float64

## Unique *MessyBrainz ID* mappings

1. Remove duplicates from `listenbrainz_msid_mapping.csv`
2. Save to `small_msid_mapping.csv`

**Unzip `listenbrainz_msid_mapping.csv.zst`**

In [ ]:
src = os.path.join(data_root, "listenbrainz_msid_mapping.csv.zst")
listenbrainz_msid_mapping_fn = shutil.copy2(src, scratch_root)
!unzstd {listenbrainz_msid_mapping_fn}

In [7]:
unique_msids = set(data['msid'])
print(f"Unique msids: {len(unique_msids)}")

Unique msids: 6300126


**Write `small_msid_mapping.csv`**

In [8]:
with open(os.path.join(scratch_root, "listenbrainz_msid_mapping.csv")) as r_fp, open(os.path.join(scratch_root, "small_msid_mapping.csv"), "w") as w_fp:
    r = csv.reader(r_fp)
    w = csv.writer(w_fp)
    header = next(r)
    w.writerow(header)
    for line in r:
        if line[0] in unique_msids and (line[2] == "exact_match" or line[2] == "high_quality"):
            w.writerow(line)

unique_msids = None

In [3]:
csv = None

**Read `small_msid_mapping.csv`**

In [7]:
smallmapping = pd.read_csv(os.path.join(scratch_root, "small_msid_mapping.csv"))
smallmapping = smallmapping.drop("match_type", axis=1)
smallmapping.set_index("recording_msid", inplace=True)
smallmapping.head()

,recording_mbid
recording_msid,
61612055-12bd-4225-8f11-b8036021d72a,66a08ebb-1d9c-4434-bd23-806dda6d2a8d
881427b4-cfa5-4223-8fa3-288aa944525c,43a21737-0462-423d-9105-c5f63d67b5b0
97c45bd5-d372-4502-add8-a8daba5e4c6e,834b5aa5-8ef0-465d-94d4-026c099d6e76
fb988713-ad8d-4a59-be17-ba04b0a1fcc0,f03e2d04-c97e-4173-a612-daa17bd3f552
8c3197d8-8965-423f-8fd7-f847a665c6d3,fc9bcf57-db4b-4d9e-9006-87576b9def0b


## Convert *User ID - MessyBrainz ID* into *User ID - MusicBrainz Recording ID*

In [8]:
recording_mbids = []
for index, row in tqdm(data.iterrows()):
    if row["msid"] in smallmapping.index:
        recording_mbid = smallmapping.loc[row["msid"]]["recording_mbid"]
    else:
        recording_mbid = None
    recording_mbids.append(recording_mbid)

smallmapping = None
data["mbid"] = recording_mbids
recording_mbids = None
data = data.drop("msid", axis=1)
data = data.dropna(subset=["mbid"])
data.head()

22412150it [17:40, 21124.50it/s]


,userid,mbid
0,21562,9c047e87-039a-43a4-9994-34940477423c
1,30309,bf4b404c-1a90-4df0-b8e7-291d54efd62e
3,33714,1712fd13-69b1-47bb-8eba-97211153c061
4,19877,873557c7-114d-4ec2-93a4-68a08c62586e
5,32780,f2d73d9b-383c-4dd1-9d79-3935bbf8d4ac


In [9]:
data.describe().loc['count']

userid    15655861.0
Name: count, dtype: float64

## Find *canonical IDs*

**Unzip `canonical_recording_redirect.csv`**

In [ ]:
src = os.path.join(data_root, "canonical_recording_redirect.csv.zst")
cannonical_msid_mapping_fn = shutil.copy2(src, scratch_root)
!unzstd {cannonical_msid_mapping_fn}

**Read `canonical_recording_redirect.csv`**

In [10]:
cannonicalmapping = pd.read_csv(os.path.join(scratch_root, "canonical_recording_redirect.csv"))
cannonicalmapping = cannonicalmapping.drop("canonical_release_mbid", axis=1)
cannonicalmapping.set_index("recording_mbid", inplace=True)
cannonicalmapping.head()

,canonical_recording_mbid
recording_mbid,
f3f8a7b8-a376-450c-8139-934d2393d49a,ecb125d7-d23e-4d76-8282-745713563110
a20f4c73-1f7a-48e7-903b-a34721c13629,ecb125d7-d23e-4d76-8282-745713563110
17344c3d-d600-4bb8-ac2d-93cab18ced4e,ecb125d7-d23e-4d76-8282-745713563110
7bf54872-7e1b-450c-9af4-385bcba33b78,ecb125d7-d23e-4d76-8282-745713563110
910f5db3-9a25-44ba-8f07-9956123c8e00,ecb125d7-d23e-4d76-8282-745713563110


### Replace `recording_mbid` with `canonical_recording_redirect.csv`

In [11]:
canonical_recording_mbids = []

for index, row in tqdm(data.iterrows()):
    if row["mbid"] in cannonicalmapping.index:
        canonical_recording_mbid = cannonicalmapping.loc[row["mbid"]]["canonical_recording_mbid"]
    else:
        canonical_recording_mbid = row["mbid"]
    canonical_recording_mbids.append(canonical_recording_mbid)

cannonicalmapping = None
data["canonical_recording_mbid"] = canonical_recording_mbids
canonical_recording_mbids = None
data = data.drop("mbid", axis=1)
data.head()

15655861it [08:44, 29832.04it/s]


,userid,canonical_recording_mbid
0,21562,9c047e87-039a-43a4-9994-34940477423c
1,30309,bf4b404c-1a90-4df0-b8e7-291d54efd62e
3,33714,1712fd13-69b1-47bb-8eba-97211153c061
4,19877,873557c7-114d-4ec2-93a4-68a08c62586e
5,32780,f2d73d9b-383c-4dd1-9d79-3935bbf8d4ac


**Write and read `userid-canonical_recording_mbid.csv`**

In [12]:
data.to_csv(os.path.join(working_root, "userid-canonical_recording_mbid.csv"), index=False)

In [6]:
data = pd.read_csv(os.path.join(working_root, "userid-canonical_recording_mbid.csv"))
data.head()

,userid,canonical_recording_mbid
0,21562,9c047e87-039a-43a4-9994-34940477423c
1,30309,bf4b404c-1a90-4df0-b8e7-291d54efd62e
2,33714,1712fd13-69b1-47bb-8eba-97211153c061
3,19877,873557c7-114d-4ec2-93a4-68a08c62586e
4,32780,f2d73d9b-383c-4dd1-9d79-3935bbf8d4ac


In [7]:
data.describe().loc['count']['userid']

15655861.0

## Get *Artist MusicBrainz IDs*

**Unzip `canonical_musicbrainz_data.csv.zst`**

In [ ]:
src = os.path.join(data_root, "canonical_musicbrainz_data.csv.zst")
cannonical_musicbrainz_data_fn = shutil.copy2(src, scratch_root)
!unzstd {cannonical_musicbrainz_data_fn}

In [8]:
shutil = None

**Read `canonical_musicbrainz_data.csv.zst`**

In [9]:
musicbrainzdata = pd.read_csv(os.path.join(scratch_root, "canonical_musicbrainz_data.csv"), usecols=["artist_mbids", "recording_mbid"])
musicbrainzdata.set_index("recording_mbid", inplace=True)
musicbrainzdata.head()

,artist_mbids
recording_mbid,
00b1a29d-ad9e-4b64-aed6-281f69f628ae,89ad4ac3-39f7-470e-963a-56509c546377
0aeea6af-3f85-45f3-88ed-8ce2bdedc4c6,89ad4ac3-39f7-470e-963a-56509c546377
24f32cf2-127e-45ca-ad19-91ed3ec87409,89ad4ac3-39f7-470e-963a-56509c546377
28e2548b-9c6f-47b7-8ab5-b1735499f291,89ad4ac3-39f7-470e-963a-56509c546377
390a9ab5-89c6-4e25-8ebf-f16a39c8c9cb,89ad4ac3-39f7-470e-963a-56509c546377


In [10]:
artist_mbids = []

for index, row in tqdm(data.iterrows()):
    if row["canonical_recording_mbid"] in musicbrainzdata.index:
        artist_mbid = musicbrainzdata.loc[row["canonical_recording_mbid"]]["artist_mbids"]
        artist_mbid = artist_mbid.split(',')[0]
    else :
        artist_mbid = None
        
    artist_mbids.append(artist_mbid)

musicbrainzdata = None
data["artist_mbids"] = artist_mbids
artist_mbids = None
data = data.drop("canonical_recording_mbid", axis=1)
data = data.dropna(subset=["artist_mbids"])
data.head()

15655861it [14:40, 17773.51it/s]


,userid,artist_mbids
0,21562,3897cf7f-9aac-4eef-aacb-ca0accdee9a2
1,30309,82cbe859-5255-4bbd-bda3-7610a3f857dc
2,33714,fbd2a255-1d57-4d31-ac11-65b671c19958
3,19877,be465d4f-c28d-4ba1-94ab-ebaada7db8af
4,32780,5673fd69-6986-4678-8f1c-519880f9f89c


In [11]:
tqdm = None

In [12]:
data.describe().loc['count']

userid    15495103.0
Name: count, dtype: float64

## Get `User ID - Artist IDs` counts

In [13]:
data = data.groupby(["userid", "artist_mbids"]).size().reset_index(name="count")
data.head()

,userid,artist_mbids,count
0,1,000fc734-b7e1-4a01-92d1-f544261b43f5,1
1,1,00eb9d25-0465-49e3-8e7a-3eacbd9ebb0d,3
2,1,012b5f71-b079-48f1-892e-af645c6576cb,1
3,1,01ce0542-1e29-4ab6-bffe-ba0e2fb61ce8,2
4,1,0383dadf-2a4e-4d10-a46a-e9e041da8eb3,1


In [14]:
data.tail()

,userid,artist_mbids,count
2119270,39526,fabb37f8-eb2a-4cc1-a72a-b56935bbb72d,23
2119271,39526,fbcd7b29-455f-49e6-9c4f-8249d20a055e,23
2119272,39526,fd429857-5ace-4609-ae54-1502c3bdac11,10
2119273,39526,fd529c0d-4a5c-479d-bbb8-601cefe2b38b,7
2119274,39526,fe125f58-6c39-42fa-85c8-3eeddda5ad21,3


In [15]:
data.to_csv(os.path.join(working_root, "userid-artist-counts.csv"), index=False)

In [16]:
data = None